# Домашнее задание 2

Условие: дана таблица с колонками (id, name, salary, managerid). Необходимо написать код на spark, который создает эту таблицу (данные указаны ниже) и в результате выдаст таблицу, в которой будут имена сотрудников, которые зарабатывают больше своих менеджеров

In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as pd

## Исходные данные:

In [2]:
data = {'id': [1,2,3,4],
        'Name': ['Joe', 'Henry', 'Sam', 'Max'],
        'Salary': ['70', '80', '60', '90'],
        'ManagerId': ['3', '4', 'Null', 'Null']}
df_pd = pd.DataFrame(data)
df_pd

,id,Name,Salary,ManagerId
0,1,Joe,70,3
1,2,Henry,80,4
2,3,Sam,60,Null
3,4,Max,90,Null


## Создаем DataFrame:

In [3]:
spark = SparkSession.builder.appName('Homework2').getOrCreate()

df = spark.createDataFrame(df_pd)
df.show()

24/04/09 17:40:59 WARN Utils: Your hostname, alexey-Aspire-A315-42 resolves to a loopback address: 127.0.1.1; using 192.168.60.84 instead (on interface wlp4s0)
24/04/09 17:40:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/09 17:40:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/09 17:41:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/home/alexey/.local/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+---+-----+------+---------+
| id| Name|Salary|ManagerId|
+---+-----+------+---------+
|  1|  Joe|    70|        3|
|  2|Henry|    80|        4|
|  3|  Sam|    60|     Null|
|  4|  Max|    90|     Null|
+---+-----+------+---------+



## Создаем вспомогательный датафрейм и соединяем:

In [4]:
managers_df = df.select('id', 'Salary').withColumnRenamed('id', 'manager_id').withColumnRenamed('Salary', 'manager_salary')
managers_df.show()

+----------+--------------+
|manager_id|manager_salary|
+----------+--------------+
|         1|            70|
|         2|            80|
|         3|            60|
|         4|            90|
+----------+--------------+



In [5]:
joined = df.join(managers_df, df['ManagerId'] == managers_df['manager_id'], 'inner')
joined.show()

+---+-----+------+---------+----------+--------------+
| id| Name|Salary|ManagerId|manager_id|manager_salary|
+---+-----+------+---------+----------+--------------+
|  1|  Joe|    70|        3|         3|            60|
|  2|Henry|    80|        4|         4|            90|
+---+-----+------+---------+----------+--------------+



## Фильтруем по условию и выводим результат:

In [6]:
joined.filter(joined['Salary'] > joined['manager_salary']).select(joined.Name).show()

+----+
|Name|
+----+
| Joe|
+----+

